Created by Finn Buchrieser, 11846398

In [ ]:

# In this script, reddit comments is preprocessed


import pandas as pd
from nltk.corpus import stopwords
import re
import nltk
from datetime import datetime

In [ ]:
# both datasets are read in using the pd.read function of pandas

comments_1 = pd.read_csv(r"C:\Users\Finn\Desktop\Foundations of CSS\Stuff\Reddit_Posts"
                         r"\wallstreetbets_comments.csv")

In [ ]:
comments_2 = pd.read_csv(r"C:\Users\Finn\Desktop\Foundations of CSS\Stuff"
                         r"\Reddit_Posts\wallstreetbets_2022.csv")

In [ ]:
# only comments are kept in the df for further use, posts and unnecessary
# columns are removed
# all comments without dates are removed and the date is converted into
# the datetime format

comments_2 = comments_2.loc[comments_2["title"] == "Comment"]
comments_2.dropna(inplace = True, subset=["timestamp"])
comments_2["timestamp"] = comments_2["timestamp"].apply(lambda x: datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S'))
comments_2 = comments_2[["id", "body", "timestamp"]]

In [ ]:
comments_1 = comments_1[["id", "body", "created_utc"]]
comments_1.dropna(inplace = True, subset=["created_utc"])
comments_1["created_utc"] = comments_1["created_utc"].apply(lambda x: datetime.fromtimestamp(x))

In [ ]:
# both datasets are combined

comments = pd.concat([comments_1, comments_1.rename({"timestamp":"created_utc"})])

In [ ]:
# stopwords, unnecessary punctuation and extremely long words
# are removed, also comments with less than 3 or more than 150
# words are removed

# Laden der Stop-Wörter aus der Standardbibliothek von NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Funktion zum Bereinigen des Texts
def clean_text(text):
    # Entfernen von "&" und "%" Zeichen
    text = re.sub(r'[&%]', '', text)
    # Tokenisierung des Textes
    tokens = text.split()
    # Entfernen von Stop-Wörtern
    tokens = [token for token in tokens if token not in stop_words]
    # Entfernen von Wörtern, die länger als 15 Zeichen sind
    tokens = [token for token in tokens if len(token) <= 15]
    # Zusammensetzen der Token zu einem Text
    cleaned_text = ' '.join(tokens)
    return cleaned_text

# Bereinigen des Textes
comments['cleaned_text'] = comments['body'].apply(lambda x: clean_text(str(x)))

# Entfernen von Posts, die länger als 150 Wörter sind
comments = comments[comments['cleaned_text'].apply(lambda x: len(x.split()) <= 150)]
comments = comments[comments['cleaned_text'].apply(lambda x: len(x.split()) >= 3)]

In [ ]:
# the data is saved in a csv fiel for further sentiment analysis

comments.to_csv(r"C:\Users\Finn\Desktop\CSS_Project"
                r"\Reddit_Posts\preprocessed.csv")